In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
import glob as glob
import matplotlib as mpl
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
import seaborn as sns

import corner
import json
import pathlib
import pickle
import utils
import warnings

from astropy import constants as const
from astropy import units as uni
from astropy.io import ascii, fits
from astropy.time import Time
from mpl_toolkits.axes_grid1 import ImageGrid

# Default figure dimensions
FIG_WIDE = (11, 5)
FIG_LARGE = (8, 11)

# Figure style
sns.set(style="ticks", palette="colorblind", color_codes=True, context="talk")
params = {
    # xticks
    "xtick.top": False,
    "xtick.direction": "out",
    "xtick.major.size": 5,
    "xtick.minor.visible": False,
    # yticks
    "ytick.right": False,
    "ytick.direction": "out",
    "ytick.major.size": 5,
    "ytick.minor.visible": False,
    # pallete
    "axes.prop_cycle": mpl.cycler(
        color=[
            "#fdbf6f",  # Yellow
            "#ff7f00",  # Orange
            "#a6cee3",  # Cyan
            "#1f78b4",  # Blue
            "#956cb4",  # Purple
            "#029e73",  # Green
            "#c44e52",  # Red
        ]
    ),
}
plt.rcParams.update(params)

<IPython.core.display.Javascript object>

In [3]:
%matplotlib qt5

<IPython.core.display.Javascript object>

## Load

In [35]:
base_dir = "data/retrievals/spot_upper_bound"

########
# Models
########
fpaths_dict = {
    "Na+K+TiO (clear)": f"{base_dir}/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO",
    "Na+K+TiO (haze)": f"{base_dir}/HATP23_E1_NoHet_FitP0_NoClouds_Haze_fitR0_Na_K_TiO",
    "Na+K+TiO (spot)": f"{base_dir}/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO",
    "Na+K+TiO (spot+haze)": f"{base_dir}/HATP23_E1_Het_FitP0_NoClouds_Haze_fitR0_Na_K_TiO",
}
fpaths_dict

{'Na+K+TiO (clear)': 'data/retrievals/spot_upper_bound/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO',
 'Na+K+TiO (haze)': 'data/retrievals/spot_upper_bound/HATP23_E1_NoHet_FitP0_NoClouds_Haze_fitR0_Na_K_TiO',
 'Na+K+TiO (spot)': 'data/retrievals/spot_upper_bound/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO',
 'Na+K+TiO (spot+haze)': 'data/retrievals/spot_upper_bound/HATP23_E1_Het_FitP0_NoClouds_Haze_fitR0_Na_K_TiO'}

<IPython.core.display.Javascript object>

In [38]:
# Load
data_dict = {
    model_name: {
        "data": ascii.read(f"{model_dir}/retr_model.txt"),
        "sampled_data": ascii.read(
            f"{model_dir}/retr_model_sampled_Magellan_IMACS.txt"
        ),
    }
    for (model_name, model_dir) in fpaths_dict.items()
}

instrument_dict = {
    "Magellan_IMACS": {
        "plot_kwargs": {
            "c": "w",
            "mec": "k",
            "fmt": "o",
            "ecolor": "k",
            "label": "Magellan/IMACS",
            "zorder": 10,
        },
        "data": ascii.read(f"{base_dir}/instruments/retr_Magellan_IMACS.txt"),
    }
}

<IPython.core.display.Javascript object>

## Plot

In [39]:
fig, ax = plt.subplots(figsize=FIG_WIDE)

for (model_name, model_dict) in data_dict.items():
    # Plot model
    p = utils.plot_model(
        ax,
        model_dict,
        model_kwargs={"label": model_name},
        fill_kwargs={"alpha": 0.25},
        sample_kwargs={"marker": "o"},
    )

#############
# Instruments
#############
for (instrument_name, instrument_data) in instrument_dict.items():
    instrument_name = instrument_name.replace("_", "/")
    utils.plot_instrument(ax, instrument_data)

####################
# Annotate Delta lnZ
####################
# fpath = f"{models[model_key_name]}/retrieval.pkl"
# fpath_flat = f'{models["K (clear)"]}/retrieval.pkl'
# DlnZ, DlnZ_unc, lnZ, lnZ_unc = utils.get_Delta_lnZ(fpath, fpath_flat)
# s = f"$\Delta \ln(Z) = {DlnZ:.2f} \pm {DlnZ_unc:.2f}$"
# ax.annotate(s, (0.02, 0.05), xycoords="axes fraction")

"""
# Plot inset
axins = ax.inset_axes([0.5, 0.5, 0.5, 0.5])
# Shortcut to zoom in on last instrument
p = plot_instrument(axins, instr, instr_sampled, instr_kwargs=configs, sampled_kwargs=sampled_kwargs)
axins.set_xlim(p.get_xlim())
axins.set_ylim(p.get_ylim())
plot_model(axins, model, model_kwargs=model_kwargs, fill_kwargs=fill_kwargs)
ax.indicate_inset_zoom(axins, alpha=1.0, edgecolor='w')
"""

ax.set_xlim(0.5, 0.95)
ax.set_ylim(12_400, 14_500)
ax.set_xlabel(r"Wavelength $(\mu\mathrm{m})$")
ax.set_ylabel("Transit depth (ppm)")
ax.legend(loc=1, ncol=3, fontsize=14)

#########
# Species
#########
species = {
    "Na I-D": 5892.9,
    #'Hα':6564.6,
    "K I_avg": 7682.0,
    "Na I-8200_avg": 8189.0,
}
[
    ax.axvline(wav / 10000, lw=0.5, ls="--", color="grey", zorder=0)
    for name, wav in species.items()
]

# ax.annotate(f"Python 2", (0.05, 0.9), xycoords='axes fraction')

# plt.savefig('/Users/mango/Desktop/exoretrievals_py2.png', dpi=250, bbox_inches='tight')

fig.set_size_inches(FIG_WIDE)
# fig.tight_layout()
# utils.savefig('projects/HATP23b/paper/figures/retrievals/retrieval.pdf')
# plt.savefig(f"/Users/mango/Desktop/tspec_haze_mid.pdf", bbox_inches="tight")
# plt.savefig(f'../retrieval/kreidberg/{species}/tspec/retr_{basename}.pdf', bbox_inches='tight')

<IPython.core.display.Javascript object>

## Corner plot

In [41]:
fpath = "data/retrievals/spot_lower_bound/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_Na_K_TiO/retrieval.pkl"
post = utils.load_pickle(fpath)

df = pd.DataFrame(post["samples"])

params = {
    "logP0": r"$\log P_0$",
    "T": r"$T_\mathrm{p}$",
    #     'logH2O':r'$\log \mathrm{H}_2\mathrm{O}$',
    "logNa": r"$\log \mathrm{Na}$",
    "logK": r"$\log \mathrm{K}$",
    "logTiO": r"$\log \mathrm{TiO}$",
    "f": r"$f$",
}

if "_Haze" in fpath:
    params["loga"] = r"$\log a$"
    params["gamma"] = r"$\gamma_\mathrm{haze}$"

if "_Het" in fpath:
    params["Tocc"] = r"$T_\mathrm{star}$"
    # params["Thet"] = r"$T_\mathrm{het}$"
    # params["Fhet"] = r"$f_\mathrm{het}$"

df_params = df[params.keys()]

corner_kwargs = {"show_titles": True}
hist_kwargs = {"histtype": "stepfilled", "lw": 2, "density": True}
fig, axes = utils.plot_corner(
    df_params,  # .drop(["Thet", "Fhet"], axis=1),
    params=params,
    c=f"C5",
    corner_kwargs=corner_kwargs,
    hist_kwargs=hist_kwargs,
)

fig.set_size_inches(18, 18)

# utils.savefig('projects/HATP23b/paper/figures/retrievals/retrieval_corner.pdf')
# plt.savefig(f'../retrieval/kreidberg/{species}/corner/corner_{basename}.pdf', bbox_inches='tight')
# plt.savefig(f'/Users/mango/Desktop/corner_{source}_all.pdf', bbox_inches='tight')
# plt.savefig(f"/Users/mango/Desktop/corner_test.pdf", bbox_inches="tight")

<IPython.core.display.Javascript object>

## Evidences

In [33]:
base_dir = "data/retrievals/spot_upper_bound"
species = ["Na", "K", "TiO", "Na_K", "Na_TiO", "K_TiO", "Na_K_TiO"]

# Load
data_dict = {
    sp: {
        "clear": utils.load_pickle(
            f"{base_dir}/HATP23_E1_NoHet_FitP0_NoClouds_NoHaze_fitR0_{sp}/retrieval.pkl"
        ),
        "haze": utils.load_pickle(
            f"{base_dir}/HATP23_E1_NoHet_FitP0_NoClouds_Haze_fitR0_{sp}/retrieval.pkl"
        ),
        "spot": utils.load_pickle(
            f"{base_dir}/HATP23_E1_Het_FitP0_NoClouds_NoHaze_fitR0_{sp}/retrieval.pkl"
        ),
        "spot+haze": utils.load_pickle(
            f"{base_dir}/HATP23_E1_Het_FitP0_NoClouds_Haze_fitR0_{sp}/retrieval.pkl"
        ),
    }
    for sp in species
}

<IPython.core.display.Javascript object>

In [34]:
evidences = {}
for species_name, species_data in data_dict.items():
    evidences[species_name] = {}
    for model_name, model_data in species_data.items():
        evidences[species_name][model_name] = model_data[
            "lnZ"
        ]  # , model_data["lnZerr"]

df_evidences = pd.DataFrame(evidences)

df_evidences

,Na,K,TiO,Na_K,Na_TiO,K_TiO,Na_K_TiO
clear,-134.326099,-136.291638,-132.379316,-134.546691,-132.471097,-132.514284,-132.845271
haze,-134.654415,-135.190496,-133.101873,-135.108784,-133.439754,-133.333176,-133.721154
spot,-133.760441,-134.894198,-131.574694,-134.154917,-131.838029,-131.994154,-132.274421
spot+haze,-134.040044,-134.300003,-132.464083,-134.458334,-132.797839,-132.943719,-133.215873


<IPython.core.display.Javascript object>

In [29]:
species_min = df_evidences.min().idxmin()
model_min = df_evidences[species_min].idxmin()
print(species_min, model_min)

K clear


<IPython.core.display.Javascript object>

In [30]:
data_flat = data_dict[species_min][model_min]

<IPython.core.display.Javascript object>

In [32]:
df_evidences - data_flat["lnZ"]
# species_max = df_retr.max()
# species_max
# model_max = df_retr[species_max].idxmax()
# species_max, model_max

,Na,K,TiO,Na_K,Na_TiO,K_TiO,Na_K_TiO
clear,1.965539,0.000000,3.912322,1.744947,3.820541,3.777354,3.446367
haze,1.637223,1.101142,3.189765,1.182854,2.851884,2.958462,2.570484
spot,2.531197,1.397440,4.716944,2.136721,4.453609,4.297484,4.017217
spot+haze,2.251594,1.991635,3.827555,1.833304,3.493799,3.347919,3.075765


<IPython.core.display.Javascript object>

\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &          Na &           K &         TiO &        Na\\_K &      Na\\_TiO &       K\\_TiO &    Na\\_K\\_TiO \\\\\n\\midrule\nclear     & -134.432372 & -136.275292 & -132.558632 & -134.628939 & -132.418799 & -132.533571 & -132.831056 \\\\\nhaze      & -134.633771 & -135.177808 & -133.159445 & -135.058690 & -133.452823 & -133.437180 & -133.748740 \\\\\nspot      & -134.654943 & -136.336323 & -132.489503 & -135.187450 & -132.778055 & -132.893856 & -133.095527 \\\\\nspot+haze & -135.154228 & -135.427309 & -133.324230 & -135.483514 & -133.648063 & -133.775133 & -134.170645 \\\\\n\\bottomrule\n\\end{tabular}\n